# load data

In [ ]:
from scipy.io import loadmat

data = loadmat('all_monkey_data/one_ff_data/sessions_python.mat',
               squeeze_me=True,
               struct_as_record=False)

sessions = data['sessions_out']


## see one trial

In [ ]:
# indices
session_num = 0
trial_num = 0

# params
prs = default_prs()

# session / behaviour
session = sessions[session_num]
behaviour = session.behaviour

# trials / stats
all_trials = behaviour.trials
all_stats = behaviour.stats
trial_ids = np.arange(len(all_trials))

trial = all_trials[trial_num]
stats = all_stats[trial_num]
pos_rel = stats.pos_rel

# continuous data
continuous = trial.continuous
print(continuous._fieldnames)

x = continuous.xmp
y = continuous.ymp
v = continuous.v
w = continuous.w
t = continuous.ts

# time step
prs.dt = round(np.mean(np.diff(t)), 5)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(x, y, 'k-')
plt.xlabel('x (forward)')
plt.ylabel('y (lateral)')
plt.axis('equal')


## neural data

In [ ]:
units = sessions[session_num].units
n_units = len(units)
 
# get unit0's trial_0 spike times
trial_neural_data = {}
for unit_id in range(len(sessions[session_num].units)):
    trial_neural_data[unit_id] = sessions[session_num].units[unit_id].trials[trial_num].tspk


# prepare data

In [ ]:
from neural_data_analysis.topic_based_neural_analysis.replicate_one_ff import one_ff_data_processing
from neural_data_analysis.topic_based_neural_analysis.replicate_one_ff import population_analysis_utils

covariate_names = [
    'v', 'w', 'd', 'phi',
    'r_targ', 'theta_targ',
    'eye_ver', 'eye_hor'
]

covariates_concat, trial_id_vec = population_analysis_utils.concatenate_covariates_with_trial_id(
    trials=all_trials,
    trial_indices=trial_ids,
    covariate_fn=lambda tr: one_ff_data_processing.compute_all_covariates(tr, prs.dt),
    time_window_fn=population_analysis_utils.full_time_window,
    covariate_names=covariate_names
)

covariates_concat['v'].shape

In [ ]:
Y = np.zeros((len(trial_id_vec), n_units))
for k in range(n_units):
    spk_counts, trial_id_vec_spk = population_analysis_utils.concatenate_trials_with_trial_id(
        all_trials,
        trial_ids,
        lambda tr, tid: population_analysis_utils.bin_spikes(
            trial_neural_data[k],
            tr.continuous.ts
        ),
        population_analysis_utils.full_time_window
    )
    Y[:, k] = spk_counts

Y_smooth = population_analysis_utils.smooth_signal(Y, prs.neural_filtwidth) / prs.dt

In [ ]:
all_events = {}
for event in ['t_targ', 't_move', 't_rew']:
    events_concat, trial_id_vec_evt = population_analysis_utils.concatenate_trials_with_trial_id(
        all_trials,
        trial_ids,
        lambda tr, tid: population_analysis_utils.event_impulse(tr, tid, event),
        population_analysis_utils.full_time_window
    )
    all_events[event] = events_concat
    
events_concat.shape

# PGAM

## temporal covariance

Temporal filters g were parameterized using a basis of ten raised cosine filters spanning a range of 600 milliseconds. The filter associated with target-onset was causal ([0, 600] ms), while the remaining filters were non-causal ([-300, 300] ms). Both spike-history filter h and coupling filter p were expressed using a basis of ten causal raised cosine filters in logarithmic time scale. Spike-history filters spanned 350 ms, while coupling filters spanned 1.375 seconds.

In [ ]:
unit_idx = 0
sm_handler = one_ff_pgam_design.build_smooth_handler_for_unit(
    unit_idx=unit_idx,                         # <-- choose the unit you want
    covariates_concat=covariates_concat,
    covariate_names=covariate_names,
    trial_id_vec=trial_id_vec,
    Y_binned=Y,
    all_events=all_events,
    dt=prs.dt,
    tuning_covariates=covariate_names,  # or a subset
    use_cyclic=set(),                  # e.g., {'heading_angle'}
    order=4,
)

sm_handler.smooths_var

# Call class

In [ ]:
binned_spikes_df = pd.DataFrame(Y, columns=np.arange(n_units))
runner = pgam_class.PGAMclass(x_var=binned_spikes_df, bin_width=prs.dt,
                                 save_dir='all_monkey_data/one_ff_data/pgam_results')

runner.sm_handler = sm_handler
runner.trial_ids = trial_id_vec
runner.train_trials = runner.trial_ids % 3 != 1
runner.run_pgam(neural_cluster_number=unit_idx)

In [ ]:
runner.kernel_h_length = 100
runner.post_processing_results()
runner.save_results()

# Build design (not using functions)

In [ ]:
data_df = pd.DataFrame(data_obj.covariates)

linear_vars = [
    'v', 'w', 'd', 'r_targ',
    'eye_ver', 'eye_hor',
]

angular_vars = [
    'phi', 'theta_targ',
]

X_tuning, tuning_meta = one_ff_glm_design.build_continuous_tuning_block(
    data=data_df,
    linear_vars=linear_vars,
    angular_vars=angular_vars,
    n_bins=10,
    center=True,
)

In [ ]:
unit_idx = 0


specs, meta = temporal_feats._init_predictor_specs(
    data_obj.prs.dt,
    data_obj.trial_ids,
)


lags, B_move = glm_bases.raised_cosine_basis(
    n_basis=10,
    t_min=-0.3,
    t_max=0.3,
    dt=meta['dt'],
)

lags, B_targ = glm_bases.raised_cosine_basis(
    n_basis=10,
    t_min=0.0,
    t_max=0.6,
    dt=meta['dt'],
)

lags, B_hist = glm_bases.raised_log_cosine_basis(
    n_basis=10,
    t_min=0.0,
    t_max=0.35,   # or 1.375
    dt=meta['dt'],
    log_spaced=True,
)

# spike history basis (log-spaced, causal)
_, B_hist = glm_bases.raised_log_cosine_basis(
    n_basis=10,
    t_min=0.0,
    t_max=0.35,
    dt=meta['dt'],
)

_, B_coup = glm_bases.raised_log_cosine_basis(
    n_basis=10,
    t_min=0.0,
    t_max=1.375,
    dt=meta['dt'],
)

specs['t_targ'] = temporal_feats.PredictorSpec(signal=data_obj.events['t_targ'], bases=[B_targ])
specs['t_move'] = temporal_feats.PredictorSpec(signal=data_obj.events['t_move'], bases=[B_move])
specs['t_rew'] = temporal_feats.PredictorSpec(signal=data_obj.events['t_rew'], bases=[B_move])
specs['spike_hist'] = temporal_feats.PredictorSpec(
    signal=data_obj.Y[:, unit_idx],   # THIS neuron’s spikes (0/1 or counts)
    bases=[B_hist],
)

# # add coupling terms later
# specs[f'cpl_{j}'] = temporal_feats.PredictorSpec(
#     signal=data_obj.Y[:, j],   # spikes from neuron j
#     bases=[B_coup],
# )



In [ ]:
# ----------------------------
# 1) Build temporal block (you already do this)
# ----------------------------
design_df, meta = temporal_feats.specs_to_design_df(
    specs,
    data_obj.covariate_trial_ids,
    edge='zero',
    add_intercept=True,
    respect_trial_boundaries=False
)

# Align and concatenate
X_tuning = X_tuning.reindex(design_df.index)
design_df = pd.concat([design_df, X_tuning], axis=1)

# Response aligned to design_df rows
unit_idx = 0
y = pd.Series(data_obj.Y[:, unit_idx], index=design_df.index).to_numpy()

# If temporal builder provides a valid-row mask, apply it to X and y
rows_mask = meta.get('valid_rows_mask', None)
if rows_mask is not None:
    design_df = design_df.loc[rows_mask]
    y = y[rows_mask]

# ----------------------------
# 2) Define lambdas (match paper defaults you pasted: f~100, g/h/p~10)
# ----------------------------
lam_f = 100.0  # tuning smoothness
lam_g = 10.0   # event kernel smoothness
lam_h = 10.0   # spike-history kernel smoothness
# lam_p = 10.0 # coupling L1/smoothness (if you add coupling later)

# ----------------------------
# 3) Build GroupSpec list (this is the critical part)
# suggested vartypes:
#   - temporal kernels: 'event'
#   - spike history: 'event'
#   - linear tunings: '1D'
#   - angular tunings: '1Dcirc' (if truly circular angles; otherwise use '1D')
# ----------------------------
groups = []

# --- temporal groups are exact from meta['groups'] you pasted ---
temporal_groups = meta['groups']  # {'t_targ': [...], 't_move': [...], 't_rew': [...], 'spike_hist': [...]}

groups.append(GroupSpec(name='t_targ', cols=temporal_groups['t_targ'], vartype='event', lam=lam_g))
groups.append(GroupSpec(name='t_move', cols=temporal_groups['t_move'], vartype='event', lam=lam_g))
groups.append(GroupSpec(name='t_rew',  cols=temporal_groups['t_rew'],  vartype='event', lam=lam_g))
groups.append(GroupSpec(name='spike_hist', cols=temporal_groups['spike_hist'], vartype='event', lam=lam_h))

# --- tuning groups come from tuning_meta['groups'] ---
# tuning_meta['groups'] maps each variable -> its column names in X_tuning
for var, cols in tuning_meta['groups'].items():
    if var in angular_vars:
        vartype = '1Dcirc'
    else:
        vartype = '1D'
    groups.append(GroupSpec(name=var, cols=cols, vartype=vartype, lam=lam_f))

# If you later add 2D variables (e.g. position x/y tiled), set vartype='2D' for that group
# If you later add coupling filters and want Laplace prior (L1), put those in l1_groups below.
